In [1]:
from multitask_dataset import SingerMultiTaskDataset, collate_fn
from torch.utils.data import DataLoader


In [2]:
train_audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/Bellou/'
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/multitask_targets.csv'

In [3]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [4]:
print(training_data[0])

{'input_values': array([ 0.03695227,  0.00765143, -0.0463584 , ...,  0.0349304 ,
       -0.04109538,  0.        ], dtype=float32), 'labels': {'Unnamed: 0': 0, 'names': 'bellou_a_1', 'singer_id': 0, 'Pitch': 294.792755848602, 'SpectralCentroid': 1988.70548799565, 'SpectralSpread': 1236.07658212378, 'SpectralSkewness': 0.667606950418554, 'SpectralKurtosis': 2.23089609708891, 'SpectralFlatness': 6.13002798571896e-05, 'SpectralCrest': 0.166798680989045, 'SpectralSlope': 8.35676790222799e-08, 'SpectralDecrease': 0.0345231147320676, 'SpectralRollOff': 3619.18975289899, 'SpectralVariation': 0.633051436428841, 'SpectralFlux': 0.0021260109600433, 'HarmonicSpectralDeviation': 0.0050759970417123, 'Tristimulus_1': 0.0668331273198761, 'Tristimulus_2': 0.375872580535276, 'Tristimulus_3': 0.545649719091416, 'HarmonicOddToEvenRatio': 1.55359376773475, 'Inharmonicity': 0.096271184919293, 'HarmonicEnergy': 0.0136912304680307, 'NoiseEnergy': 0.0129354341375336, 'Noisiness': 0.464518212827659, 'HarmonicTo

In [5]:
dataloader = DataLoader(training_data, batch_size=8, shuffle=True)

In [6]:
b = next(iter(dataloader))

RuntimeError: stack expects each tensor to be equal size, but got [25538] at entry 0 and [29444] at entry 1